<a href="https://colab.research.google.com/github/Rajat-Yd/Rag_Intern_apllication/blob/main/RAG_Chatbot_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers sentence-transformers faiss-cpu pandas
!pip install streamlit
!pip install pyngrok

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd

In [ ]:
model_name = "all-MiniLM-L6-v2"  # Lightweight sentence-transformer model
embedder = SentenceTransformer(model_name)

embeddings = embedder.encode(data["answer"].tolist(), convert_to_tensor=True)


In [ ]:
import numpy as np

# Convert embeddings to NumPy and create an index
faiss_index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance metric
faiss_index.add(np.array(embeddings.cpu()))

# Save questions for reference
questions = data["question"].tolist()


In [ ]:
def retrieve_answer(query, k=1):
    query_embedding = embedder.encode([query], convert_to_tensor=False)
    distances, indices = faiss_index.search(np.array(query_embedding), k)
    return [(questions[i], data['answer'][i]) for i in indices[0]]


In [ ]:
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [ ]:
def generate_response(query):
    retrieved_answer = retrieve_answer(query)[0][1]
    input_text = f"question: {query} answer: {retrieved_answer}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(**inputs, max_length=150)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
while True:
    user_query = input("Ask a question: ")
    if user_query.lower() == "exit":
        break
    print("Answer:", generate_response(user_query))


In [ ]:
import streamlit as st

st.title("RAG Chatbot with Hugging Face Models")
query = st.text_input("Ask your question:")
if query:
    response = generate_response(query)
    st.write(f"Answer: {response}")

In [ ]:
!npx localtunnel --port 8501


In [ ]:
!curl https://loca.lt/mytunnelpassword